# Phân đoạn Pneumothorax (tràn khí màng phổi)

## Tổng quan bài toán

Trong dự án này, sẽ phải phân đoạn được đâu là phần tràn khí màng phổi từ hình ảnh X-quang đã được gán nhãn trước.

Dữ liệu bao gồm có 12955 hình X-quang từ 12047 bệnh nhân và nhãn tương ứng (có những hình ảnh có nhãn, có những hình ảnh không có nhãn tức là không bị chứng tràn khí màng phổi).

## Các bước thực hiện dự án

### 1. Phân tích và khám phá dữ liệu (EDA)

Phần đầu tập trung vào việc phân tích và khám phá dữ liệu để có thể hiểu và mô tả được nội dung và bản chất của dữ liệu.

EDA rất quan trọng khi nó có thể giúp tăng độ chính xác mô hình nếu phân tích dữ liệu tốt và có thể làm báo cáo về sau. Một số thông tin mà lúc làm EDA có thể làm đó là:
* Dữ liệu nhân khẩu học của bệnh nhân:
        * Giới tính
        * Tuổi
        * ... các thông tin này là có sẵn
* X-ray view (góc chụp của hình ảnh X-quang)
* Các thông tin:
        * Số lượng các ca bệnh mắc tràn khí phổi
        * Số lượng các ca bệnh không bị mắc tràn khí phổi
* Sự phân bố các ca bệnh theo:
        * Độ tuổi
        * Giới tính
        * ...
* Đánh giá mức pixel giữa trạng thái khỏe mạnh và trạng thái bị bệnh trong bộ dữ liệu (ví dụ: biểu đồ các giá trị cường độ) và so sánh phân bố giữa các loại này

### 2. Xây dựng và đào tạo mô hình

**Training and Validation Datasets**

Từ các phát hiện trong phần EDA, hãy sắp xếp các tập train và validation sao cho phù hợp để có thể phân đoạn được bệnh. Hãy chắc chắn xem xét các điều kiện sau:
* Thông tin nhân khẩu học, vị trí chụp hình ảnh, và số lượng hình ảnh theo độ tuổi và giới tính, ...
* Phân bố giữa các trường hợp dương tính và âm tính trong mỗi tập dữ liệu

**Kiến trúc mô hình**:

Trong dự án này, có thể sử dụng việc tinh chỉnh kiến trúc Unet3D để có thể phân đoạn được vị trí tràn khí màng phổi. Việc tinh chỉnh có thể đến từ việc đóng băng một số lớp đầu và đào tạo các lớp phía sau.

**Xử lý trước và tăng cường dữ liệu**:

Tiền xử lý trước dữ liệu trước khi đưa vào mạng để đào tạo và xác thực. Điều này có tác dụng giúp phù hợp với kiến trúc mô hình hơn, dễ đào tạo hơn để có thể tăng được hiệu suất của mô hình. Ngoài ra, có thể tăng cường dữ liệu và phải chú ý đến các thông số phản ánh được sự khác biệt có thể đến từ thế giới thực có thể nhận được khi chụp hình ảnh X-Quang.

**Đào tạo**:

Trong quá trình đào tạo, có nhiều tham số cần tinh chỉnh bao gồm:
* Các tham số trong quá trình tăng cường dữ liệu
* Batch_size
* Learning rate
* Các tham số của từng lớp cụ thể trong mạng

Cần phải mô tả được các phương pháp và tham số đã sử dụng sau khi đánh giá hiệu suất của mô hình.

### 3. Tích hợp quy trình làm việc lâm sàng

Dữ liệu hình ảnh được cung cấp để thực hiện đào tạo mô hình đang trong dạng DICOM. Trong dự án này, hãy tạo một DICOM wrapper có thể nhận các tệp DICOM chuẩn, và xuất dữ liệu ở định dạng mà mô hình có thể chấp nhận được.

# Thực hiện dự án

## Khám phá và phân tích dữ liệu (EDA)

EDA là một bước rất quan trọng giúp hiểu được bản chất của dữ liệu từ đó có thể giúp quá trình tiếp cận vấn đề và đưa ra hướng giải quyết được đúng và chính xác. Ví dụ, EDA trong dự án này có thể giúp phát hiện ra được các biểu hiện của tràn khí màng phổi trong tự nhiên. Ví dụ như: tần suất xuất hiện là bao nhiêu, độ tuổi chịu ảnh hưởng của loại bệnh này, ... 

Các việc cần EDA trên tập dữ liệu này bao gồm có:
* Dữ liệu về nhân khẩu học như giới tính, tuổi, vị trí đặt bệnh nhân, ...
* Các chế độ khi chụp ảnh X-Quang (vị trí chụp, ... )
* Số trường hợp:
    * Số lượng trường hợp bị tràn khí màng phổi
    * Số lượng trường hợp không bị tràn khí màng phổi
* Sự phân bố của các trường hợp tràn khí màng phổi đối với độ tuổi
* Sự phân bố của các trường hợp tràn khí màng phổi đối với giới tính
* Đánh giá mức pixel của dữ liệu hình ảnh cho các trạng thái khỏe mạnh và bệnh tật cần quan tâm (ví dụ như biểu đồ về các giá trị cường độ và so sánh sự phân bố này).

Ngoài ra, sau khi đã thực hiện EDA, hãy mô tả cách thiết lập tập training và testing cho bộ dữ liệu.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

##Import any other packages you may need here
import matplotlib.image as image
import os
import glob

import pydicom

In [ ]:
def show_dcm_info(dataset):
    storage_type = dataset.SOPClassUID
    
    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    
    pat_id = dataset.PatientID
    pat_age = dataset.PatientAge
    pat_sex = dataset.PatientSex
    modality = dataset.Modality
    body_part_examined = dataset.BodyPartExamined
    view_position = dataset.ViewPosition
    rows = None
    cols = None
    pixel_spacing = None
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        if 'PixelSpacing' in dataset:
            pixel_spacing = dataset.PixelSpacing
    
    meta_data = {
        "storage_type": storage_type,
        "display_name": display_name,
        "pat_age": pat_age,
        "pat_sex": pat_sex,
        "modality": modality,
        "body_part_examined": body_part_examined,
        "view_position": view_position,
        "rows": rows,
        "cols": cols,
        "pixel_spacing_x": pixel_spacing[0],
        "pixel_spacing_y": pixel_spacing[1]
    }
    
    return meta_data 

def plot_pixel_array(dataset, figsize=(10,10)):
    plt.figure(figsize=figsize)
    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    plt.show()

In [ ]:
path_csv = "../input/siim-acr-pneumothorax-segmentation-data/train-rle.csv"
df = pd.read_csv(path_csv)
df.head()

In [ ]:
file_path = "../input/siim-acr-pneumothorax-segmentation-data/dicom-images-test/1.2.276.0.7230010.3.1.2.8323329.5797.1517875190.762693/1.2.276.0.7230010.3.1.3.8323329.5797.1517875190.762692/1.2.276.0.7230010.3.1.4.8323329.5797.1517875190.762694.dcm"
dataset = pydicom.dcmread(file_path)
meta_data = show_dcm_info(dataset)
meta_data

In [ ]:
# from tqdm.notebook import tqdm

# new_df = pd.DataFrame({
#     "ImageId": [],
#     "EncodedPixels": [],
#     "storage_type": [],
#     "display_name": [],
#     "pat_age": [],
#     "pat_sex": [],
#     "modality": [],
#     "body_part_examined": [],
#     "view_position": [],
#     "rows": [],
#     "cols": [],
#     "pixel_spacing_x": [],
#     "pixel_spacing_y": []
# })

# file_path = "../input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/*/*/*.dcm"

# file_paths = glob.glob(file_path)

# for index, row in tqdm(df.iterrows()):
    
#     index_ = list(filter(lambda x: row[0] in file_paths[x], range(len(file_paths))))
#     dataset = pydicom.dcmread(file_paths[index_[0]])
#     meta_data = show_dcm_info(dataset)

#     data = {
#         "ImageId": file_paths[index_[0]].split("/")[4],
#         "EncodedPixels": row[1],
#         "storage_type": meta_data["storage_type"],
#         "display_name": meta_data["display_name"],
#         "pat_age": meta_data["pat_age"],
#         "pat_sex": meta_data["pat_sex"],
#         "modality": meta_data["modality"],
#         "body_part_examined": meta_data["body_part_examined"],
#         "view_position": meta_data["view_position"],
#         "rows": meta_data["rows"],
#         "cols": meta_data["cols"],
#         "pixel_spacing_x": meta_data["pixel_spacing_x"],
#         "pixel_spacing_y": meta_data["pixel_spacing_y"]
#     }
    
#     new_df = new_df.append(data, ignore_index=True)
    

In [ ]:
# new_df.to_csv('train_info.csv', header=True, index=False)

In [ ]:
all_xray_df = pd.read_csv('../input/segment-data/train_info.csv')
all_xray_df.sample(3)

In [ ]:
all_xray_df.info()

### Phân tích phân phối của các trường thuộc tính

#### Age

In [ ]:
h = all_xray_df['pat_age'].hist(bins=100)

In [ ]:
over100 = all_xray_df[all_xray_df['pat_age'] > 100]
print(len(over100))
over100

Có 2 records chứa giá trị tuổi trên 100. Đây rõ ràng là nhầm lẫn, cách đơn giản là đặt nó về giá trị 100, vì điều này ít nhất cũng cho hai bản ghi này nằm về cùng một giá trị có thể.

In [ ]:
all_xray_df['pat_age'] = all_xray_df.apply(lambda x: 101 if x['pat_age'] > 100 else x['pat_age'], axis=1)

In [ ]:
over100 = all_xray_df[all_xray_df['pat_age'] > 100]
print(len(over100))
over100

In [ ]:
h = all_xray_df['pat_age'].hist(bins=100)

Phân phối tuổi của bệnh nhân gần giống phân phối chuẩn với việc hơi lệch phải một chút, đỉnh của phân phối cao nhất là tầm 58 tuổi.

In [ ]:
all_xray_df[all_xray_df['pat_age'] <= 100][['pat_age']].describe()

Khoảng dao động của tuổi bệnh nhân nằm trong khoảng từ 1 cho đến 94.

#### Gender

In [ ]:
gender_m = all_xray_df[all_xray_df['pat_sex'] == 'M']
gender_f = all_xray_df[all_xray_df['pat_sex'] == 'F']

In [ ]:
plt.bar(['Male','Female'], [len(gender_m), len(gender_f)])

In [ ]:
print(f'Patien Gender distribution\nMale: {len(gender_m)} ({100.0*len(gender_m)/len(all_xray_df):.2f}%), Female: {len(gender_f)} ({100.0*len(gender_f)/len(all_xray_df):.2f}%)')

Số lượng về giới tính của bệnh nhân cũng không bị bias quá nhiều về lớp nào cả, cả hai lớp này khá đồng đều (tương đối).

#### View Position

In [ ]:
all_xray_df['view_position'].unique()

In [ ]:

pos_pa = all_xray_df[all_xray_df['view_position'] == 'PA']
pos_ap = all_xray_df[all_xray_df['view_position'] == 'AP']
plt.bar(['PA','AP'], [len(pos_pa), len(pos_ap)])


In [ ]:
print(f'View position value distribution\nPA: {len(pos_pa)} ({100.0*len(pos_pa)/len(all_xray_df):.2f}%), AP: {len(pos_ap)} ({100.0*len(pos_ap)/len(all_xray_df):.2f}%)')

Cảm giác hai lớp này có sự bias nhẹ! Thiên về lớp PA

#### Số lượng bệnh/không bị bệnh

In [ ]:
finding = all_xray_df[all_xray_df['EncodedPixels'] != ' -1']
no_finding = all_xray_df[all_xray_df['EncodedPixels'] == ' -1']

In [ ]:
print(f'Number of sick value distribution\nfinding: {len(finding)} ({100.0*len(finding)/len(all_xray_df):.2f}%), no finding: {len(no_finding)} ({100.0*len(no_finding)/len(all_xray_df):.2f}%)')

In [ ]:
plt.bar(['finding','no finding'], [len(finding), len(no_finding)])

Có thể thấy đã xuất hiện hiện tượng bị bias. Số lượng no finding nhiều hơn rất nhiều so với số lượng finding. Cần phải giải quyết vấn đề mất cân bằng này. Một trong nhưng cách đó là:
* Giảm bớt số lượng no-finding về gần với finding
* Áp dụng các kĩ thuật tăng cường dữ liệu đối với lớp finding

#### Patient ID

In [ ]:
unique_patients_num = all_xray_df['display_name'].nunique()
print(f'Total unique patients: {unique_patients_num}, average number records per patient: {len(all_xray_df)/unique_patients_num :.2f}')

In [ ]:
records_per_patient = []
for pid in all_xray_df['display_name'].unique():
    records_per_patient.append(len(all_xray_df[all_xray_df['display_name'] == pid]))

In [ ]:
unique, counts = np.unique(np.array(records_per_patient), return_counts=True)
plt.bar(unique, counts)

In [ ]:
def num_patients_with_records(num):
    return (np.array(records_per_patient) == num*np.ones(len(records_per_patient))).sum()

In [ ]:
for i in range(1,11):
    print(f'Number of patients with {i} records in the dataset: {num_patients_with_records(i)} ({100.0*num_patients_with_records(i)/unique_patients_num :.2f})')

Số lượng 1 bệnh nhân chỉ có một records tương ứng với việc một bệnh nhân chỉ có 1 phát hiện segment là nhiều nhất. Còn lại, số lượng 2, 3, ...  phân đoạn segment thì ít hơn và có trường hợp outline là có 10 phân đoạn segment cho một bệnh  Tuy nhiên tập trung nhiều vào trường hợp 1 nên cũng không cần quá quan tâm đến hiện tượng bias ở đây, hoặc nếu muốn xử lý thì chỉ nên xử lý phần có 2 segment.

#### Image width & Height, pixel spacing

In [ ]:
desc = all_xray_df.describe()
desc

In [ ]:
def show_min_max_values(col, name):
    print(f'{name} range: [{desc[col]["min"]}, {desc[col]["max"]}]')

In [ ]:
show_min_max_values('rows', 'Image Width')
show_min_max_values('cols', 'Image Height')

In [ ]:
show_min_max_values('pixel_spacing_x', 'Pixel Spacing over X')
show_min_max_values('pixel_spacing_y', 'Pixel Spacing over Y')

Kích thước hình ảnh cũng như pixel spacing đối với các hình ảnh khác nhau không khác biệt, nên phần này không cần xử lý trước.

#### Kiểm tra rage age nằm trong khoảng nào đối với bệnh nhân bị bệnh

In [ ]:
finding = all_xray_df[all_xray_df['EncodedPixels'] != ' -1']
no_finding = all_xray_df[all_xray_df['EncodedPixels'] == ' -1']

In [ ]:
finding[finding['pat_age'] <= 100][['pat_age']].describe()

In [ ]:
no_finding[no_finding['pat_age'] <= 100][['pat_age']].describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figheight(5)
fig.set_figwidth(10)
ax1.title.set_text("Range Age For Finding")
ax1.hist(finding['pat_age'], bins = 100)
ax2.title.set_text("Range Age For No Finding")
ax2.hist(no_finding['pat_age'], bins = 100)
plt.show()

Có thể thấy khoảng độ tuổi mắc tràn khí phổi trải gần đều từ 19 cho đến 63 tuổi, trong khi đối với những trường hợp không bị mắc thì lại tập trung vào khoảng 58 tuổi.

#### Kiểm tra phân bố giới tính với bị bệnh/ko bị bệnh

In [ ]:
gender_m = finding[finding['pat_sex'] == 'M']
gender_f = finding[finding['pat_sex'] == 'F']
plt.bar(['M','F'], [len(gender_m), len(gender_f)])
print(f'Patien Gender distribution\nMale: {len(gender_m)} ({100.0*len(gender_m)/len(all_xray_df):.2f}%), Female: {len(gender_f)} ({100.0*len(gender_f)/len(all_xray_df):.2f}%)')

In [ ]:
gender_m = no_finding[no_finding['pat_sex'] == 'M']
gender_f = no_finding[no_finding['pat_sex'] == 'F']
plt.bar(['M','F'], [len(gender_m), len(gender_f)])
print(f'Patien Gender distribution\nMale: {len(gender_m)} ({100.0*len(gender_m)/len(all_xray_df):.2f}%), Female: {len(gender_f)} ({100.0*len(gender_f)/len(all_xray_df):.2f}%)')

Phân bố theo giới tính có vẻ khá đồng đều, không bị bias quá về lớp nào cả.

#### Kiểm tra phân bố theo view position với bị bệnh/ko bị bệnh

In [ ]:
pos_pa = finding[finding['view_position'] == 'PA']
pos_ap = finding[finding['view_position'] == 'AP']
plt.bar(['PA','AP'], [len(pos_pa), len(pos_ap)])
print(f'View position value distribution\nPA: {len(pos_pa)} ({100.0*len(pos_pa)/len(finding):.2f}%), AP: {len(pos_ap)} ({100.0*len(pos_ap)/len(finding):.2f}%)')

In [ ]:

pos_pa = no_finding[no_finding['view_position'] == 'PA']
pos_ap = no_finding[no_finding['view_position'] == 'AP']
plt.bar(['PA','AP'], [len(pos_pa), len(pos_ap)])
print(f'View position value distribution\nPA: {len(pos_pa)} ({100.0*len(pos_pa)/len(no_finding):.2f}%), AP: {len(pos_ap)} ({100.0*len(pos_ap)/len(no_finding):.2f}%)')

Đối với trường hợp có bệnh, view position có vẻ hơi bị bias một chút về lớp PA, trong khi đó, những trường hợp không bị bệnh, việc bias này không đáng kể.

#### Phân tích theo mức pixel

In [ ]:
sample_df = all_xray_df.sample(1000)
sample_df.head(3)

In [ ]:
finding_sample = sample_df[sample_df['EncodedPixels'] != ' -1']
no_finding_sample = sample_df[sample_df['EncodedPixels'] == ' -1']

In [ ]:
finding_sample_1 = finding_sample.iloc[0]
finding_sample_1

In [ ]:
fordel_path = "../input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/"
file_path = os.path.join(fordel_path, finding_sample_1.ImageId) + "/*/*.dcm"
image_path = glob.glob(file_path)[0]

In [ ]:
def show_image_distr(data):
    data = data/255.
    f = plt.figure()
    f.set_figwidth(10)

    s1 = f.add_subplot(1, 2, 1)
    s1.set_title('Image')
    plt.imshow(data, cmap='gray')


    s2 = f.add_subplot(1, 2, 2)
    s2.set_title('Intensity Distribution')
    plt.hist(data.ravel(), bins = 256)

    plt.show()

In [ ]:
dataset = pydicom.dcmread(image_path)
img_data = dataset.pixel_array
show_image_distr(img_data)

In [ ]:
def get_image_data(row):
    fordel_path = "../input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/"
    file_path = os.path.join(fordel_path, row["ImageId"]) + "/*/*.dcm"
    image_path = glob.glob(file_path)[0]
    if image_path is not None:
        dataset = pydicom.dcmread(image_path)
        img_data = dataset.pixel_array
        img_data = img_data/255.
        return img_data
    else:
        return None

In [ ]:
finding_sample['image_data'] = finding_sample.apply(get_image_data, axis=1)
finding_sample.head(2)

In [ ]:
finding_data = finding_sample[finding_sample['image_data'].notna()]['image_data'].values
finding_data.shape

In [ ]:
finding_intensities = []
for data in finding_data:
    finding_intensities.extend(data.flatten().tolist())

In [ ]:
plt.hist(finding_intensities[:1000000],bins=256)
plt.show()

#### Sample finding distribution with background

In [ ]:
for data in finding_data[:10]:
    show_image_distr(data)

Hình ảnh gốc đang bị ảnh hưởng bởi 2 yếu tố:
* Quá nhiều nền
* Quá nhiều phần hình ảnh bị chặn bởi các vật thể khác (xương, nhựa, ... ) dẫn đến cường độ màu rất sáng

#### Remove Background

In [ ]:
def show_image_mask(img_data, th):
    img_data = img_data/255.
    f = plt.figure()
    f.set_figwidth(15)

    s1 = f.add_subplot(1, 3, 1)
    s1.set_title('Image')
    plt.imshow(img_data, cmap='gray')

    mask = data > th

    s2 = f.add_subplot(1, 3, 2)
    s2.set_title('Mask')
    plt.imshow(mask.astype(int)*255, cmap='gray')
    
    s3 = f.add_subplot(1, 3, 3)
    s3.set_title('Intensity Distribution')
    plt.hist(img_data[mask], bins = 256)

    plt.show()

In [ ]:
background_threshold = 0.1

for data in finding_data[:10]:
    show_image_mask(data, background_threshold)

#### Cần loại bỏ các vùng trắng quá mức

In [ ]:
foreground_threshold = 0.9

In [ ]:
def get_intensities(option="finding"):
    fordel_path = "../input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/"
    
    intensities = []
    if option == "finding":
        image_ids = sample_df[sample_df['EncodedPixels'] != ' -1'][:100].ImageId
    else:
        image_ids = sample_df[sample_df['EncodedPixels'] == ' -1'][:100].ImageId
    for image_id in image_ids:
        file_path = os.path.join(fordel_path, image_id) + "/*/*.dcm"
        image_path = glob.glob(file_path)[0]
        dataset = pydicom.dcmread(image_path)
        data = dataset.pixel_array / 255.
        mask = (data > background_threshold) & (data < foreground_threshold)
        intensities.extend(data[mask].flatten().tolist())
    return intensities
    
plt.hist(get_intensities("finding"), bins = 256)
plt.show()

#### So sánh phân phối hình ảnh bị tràn khí phổi vs hình ảnh không bị tràn khí phổi

In [ ]:
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(20)

s1 = f.add_subplot(1, 2, 1)
s1.set_title('Image Finding')
plt.hist(get_intensities("finding"), bins = 256)
plt.show()

s2 = f.add_subplot(1, 2, 2)
s2.set_title('Image No Finding')
plt.hist(get_intensities("no_finding"), bins = 256)
plt.show()


#### Kết luận

* Hình ảnh trong tập dữ liệu cần phải chuẩn hóa về đoạn từ 0 đến 1
* Background cần phải được loại bỏ với ngưỡng là > 0.1
* Foreground mà có giá trị quá sáng cũng cần phải loại bỏ, ngưỡng gợi ý là < 0.9. Đây có thể là những thực thể bên ngoài, hấp thụ tia X tốt, ví dụ như mảnh nhựa, ... )
* Phân bố hình ảnh liên quan đến độ tuổi và giới tính tương đối đồng đều, không quá bị bias về lớp đối tượng nào cả.
* Bias về số lượng hình ảnh finding và non-finding, với trường hợp non-finding có rất nhiều hình ảnh:
    * Loại bỏ các hình ảnh non-finding sao cho gần bằng với số lượng hình ảnh finding:
        * finding: 3286 (28.37%), no finding: 8296 (71.63%)
    * Tăng cường dữ liệu đối với hình ảnh finding
* View position đối với finding cũng bị bias nhẹ (có thể xử lý sau):
    * PA: 2131 (64.85%), AP: 1155 (35.15%)
* Ngoài ra còn có một số nhiễu trong hình ảnh cũng cần phải xử lý trước (xử lý sau - sau khi có base model)

# Chia dữ liệu thành tập train và tập test

Tiêu chí chia tập dữ liệu:
* Chia theo số lượng finding:
    * Lấy khoảng 3286 finding và chọn random 3286 non-finding
    * Áp dụng các kĩ thuật tăng cường dữ liệu cho finding và non-finding
* Đối với việc bị thiên lệch về view-position có thể tính sau

In [ ]:
# Read Origin Image
all_xray_df = pd.read_csv('../input/segment-data/train_info.csv')
all_xray_df.sample(3)

In [ ]:
finding = all_xray_df[all_xray_df['EncodedPixels'] != ' -1']
no_finding = all_xray_df[all_xray_df['EncodedPixels'] == ' -1'].sample(len(finding))

In [ ]:
number_train = int(0.9 * len(finding))
training_df = finding.sample(number_train)
validation_df = finding.drop(training_df.index)


In [ ]:
training_df.to_csv('train_info_split.csv', header=True, index=False)
validation_df.to_csv('val_info_split.csv', header=True, index=False)